# Proyecto inicial
Este notebook contiene un ejemplo de importación de pandas y muestra un DataFrame pequeño.

In [ ]:
import pandas as pd

# Crear un DataFrame de ejemplo
df = pd.DataFrame({'col1': [1, 2, 3], 'col2': ['a', 'b', 'c']})

# Mostrar el DataFrame (en Jupyter basta con dejar la variable en la última línea)
df